# Data Cleansing

## Import necessary libraries

In [1]:
# Add current directory to Python path for imports
import os
import sys

# Add the parent directory (project root) to Python path so we can import from src
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import numpy as np
import pandas as pd
import yaml

## Loading Datasets

In [3]:
# Load data using configuration file
configs_path = os.path.join("..", "configs", "configs.yaml")

with open(configs_path, encoding="utf-8") as config_file:
    config = yaml.safe_load(config_file)
    data_filename = config["data"]["raw_data_path"]

# Make path relative to notebook location
data_path = os.path.join("..", data_filename)

df = pd.read_csv(data_path)

print(f"Config loaded from: {configs_path}")
print(f"Data loaded successfully from: {data_path}")
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

print("First few rows:")
display(df.head(3))

Config loaded from: ..\configs\configs.yaml
Data loaded successfully from: ..\data/raw/bangkok_traffy.csv
DataFrame shape: (787026, 16)
Columns: ['ticket_id', 'type', 'organization', 'comment', 'photo', 'photo_after', 'coords', 'address', 'subdistrict', 'district', 'province', 'timestamp', 'state', 'star', 'count_reopen', 'last_activity']
First few rows:


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00


---

## Apply Transformers

In [4]:
from src import ProvinceTransformer

pt = ProvinceTransformer(columns=["province"])
df_transformed = pd.DataFrame(pt.fit_transform(df))
filtered_values = pt.get_filtered_values()

print(f"Filtered values (not found in whitelist): {filtered_values}")

df_province_count = pd.DataFrame(df_transformed.province.value_counts())
df_province_count.head(10)

Loading province whitelist from: c:\Users\pun\Desktop\CU\CEDT-Y2-S1\2110403 - Introduction to Data Science and Data Engineering\CEDT-2110403-DSDE-Project\configs\thailand_province_whitelist.json
Filtered values (not found in whitelist): ['Tillabéri', '', 'Lac', 'Perak', 'Borno', 'Riau', 'nan', 'Steiermark', 'Sahel']
Filtered values (not found in whitelist): ['Tillabéri', '', 'Lac', 'Perak', 'Borno', 'Riau', 'nan', 'Steiermark', 'Sahel']


,count
province,
กรุงเทพมหานคร,785662
นนทบุรี,261
สมุทรปราการ,244
ปทุมธานี,110
สมุทรสาคร,45
นครปฐม,30
นครราชสีมา,21
ภูเก็ต,14
เพชรบุรี,13


In [5]:
from src import DistrictSubdistrictTransformer

dst = DistrictSubdistrictTransformer(
    district_column="district", subdistrict_column="subdistrict"
)
df_transformed = pd.DataFrame(dst.fit_transform(df_transformed))

df_area_count = pd.DataFrame(df_transformed[["district", "subdistrict"]].value_counts())
df_area_count.head(10)

,,count
district,subdistrict,
เขต สวนหลวง,แขวง สวนหลวง,19663
เขต ดินแดง,แขวง ดินแดง,19477
เขต พญาไท,แขวง สามเสนใน,14205
เขต บางกะปิ,แขวง หัวหมาก,14198
เขต คลองเตย,แขวง คลองเตย,13565
เขต ประเวศ,แขวง คลองสามประเวศ,13020
เขต บางเขน,แขวง แสมดำ,12759
เขต จตุจักร,แขวง จอมพล,12719
เขต บางเขน,แขวง อนุสาวรีย์,12659


---